##Feature Extraction using RANSAC

In [2]:
!pip install opencv-python


In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [4]:
# Load the images
img1_color=cv2.imread('./left.jpg')# Query image
img2_color=cv2.imread('./right.jpg')# train image 

In [5]:
# Convert to grayscale
img1=cv2.cvtColor(img1_color,cv2.COLOR_BGR2GRAY)
img2=cv2.cvtColor(img2_color,cv2.COLOR_BGR2GRAY)

In [6]:
# Initialize the SIFT detectoe
sift=cv2.SIFT_create()

In [7]:
# Detect the keypoints and compute desciptors
kp1, des1 =sift.detectAndCompute(img1, None)
kp2, des2 =sift.detectAndCompute(img2, None)

In [8]:
# FLANN parameters and matcher setup
FLANN_INDEX_KDTREE=1
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
search_params = dict(checks=50)

In [9]:
flann=cv2.FlannBasedMatcher(index_params, search_params)

In [10]:
# Match Descriptors using KNN
matches=flann.knnMatch(des1, des2, k=2)

In [11]:
# Ratio Test to keep good matches
good=[]
for m, n in matches:
    if m.distance<0.7*n.distance:
        good.append(m)

In [12]:
# Extracting locations of good matches
points1 = np.zeros((len(good), 2), dtype=np.float32)
points2 = np.zeros((len(good), 2), dtype=np.float32)

for i, match in enumerate(good):
    points1[i,:] = kp1[match.queryIdx].pt
    points2[i,:] = kp2[match.trainIdx].pt

In [22]:
# find the homography using RANSAC, only if there are enough points
if len(points1) >= 4 and len(points2) >= 4:
    H, status = cv2.findHomography(points1, points2, cv2.RANSAC, 5.0)
else:
    print("Not enough points to calculate homography.")
    status = None


Not enough points to calculate homography.


In [18]:
#create new image thatputs two images side by side
height = max(img1_color.shape[0], img2_color.shape[0])
width = img1_color.shape[1] + img2_color.shape[1]
output = np.zeros((height, width, 3), dtype=np.uint8)

In [19]:
#place both images withing this new image
output[0:img1_color.shape[0], 0:img1_color.shape[1]] = img1_color
output[0:img2_color.shape[0], img1_color.shape[1]:img1_color.shape[1]+img2_color.shape[1]] = img2_color

In [24]:
# Draw Lines Between the matchinh points
for i, (m,color) in enumerate(zip(good, np.random.randint(0,255,(len(good),3)))):
    if status[i]:
        plt1=tuple(map(int,kp1[m.queryIdx].pt))
        plt2=tuple(map(int,kp2[m.trainIdx].pt))
        pt2=(pt2[0]+img1_color.shape[1],pt2[1])
                              
        cv2.line(output, pt1, pt2, color.tolist(), 2)

# convert the result to RGB for matplotlib display and show the final image 
output_rgb = cv2.cvtcolor(output, cv2.COLOR_BGR2RGB)

# use matplotlib to display the image
plt.figure(figsize=(15, 5))
plt.imshow(output_rgb)
plt.axis('off')
plt.title('Feature Matching with RANSAC')
plt.show()

TypeError: 'NoneType' object is not subscriptable

Not enough matching points to calculate homography.
